In [2]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import torch.optim as optim
import matplotlib.pyplot as plt

In [41]:
import os
import numpy as np
import cv2
from PIL import Image
from tqdm import tqdm

REBUILD_DATA = True

In [59]:
class DogvCat:
    def __init__(self):
        self.IMG_SIZE = 50
        self.CATS = os.path.join('data', 'training_set', 'cats')
        self.DOGS = os.path.join('data', 'training_set', 'dogs')
        self.LABELS = {self.CATS: 0, self.DOGS: 1}
    
        self.training_data = []
        self.cat_count = 0
        self.dog_count = 0

    def make_training_data(self):
        for label in self.LABELS:
            print(label)
            # for _, _, img_files in os.walk(label):
            #     for file in img_files:
            for img_files in tqdm(os.listdir(label)):
                path = os.path.join(label, img_files)
                # img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                # img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
                img = Image.open(path)
                transform = transforms.Compose([
                    transforms.Grayscale(),
                    transforms.Resize((self.IMG_SIZE, self.IMG_SIZE)),
                ])
                img = transform(img)
                self.training_data.append([np.array(img), np.eye(2)[self.LABELS[label]]])

                if label == self.CATS:
                    self.cat_count += 1
                elif label == self.DOGS:
                    self.dog_count += 1
                
        # np.random.shuffle(self.training_data)
        # np.save("training.npy", self.training_data)
        print(f'Cats: {self.cat_count}')
        print(f'Dogs: {self.dog_count}')
        print(self.training_data[0])
        

In [60]:
if REBUILD_DATA:
    obj = DogvCat()
    obj.make_training_data()

data\training_set\cats


100%|█████████████████████████████████████████████████████████████████████████████| 4000/4000 [00:11<00:00, 345.46it/s]


data\training_set\dogs


100%|█████████████████████████████████████████████████████████████████████████████| 4005/4005 [00:11<00:00, 338.98it/s]

Cats: 4000
Dogs: 4005
[array([[ 41,  47,  42, ...,  93, 107, 179],
       [ 40,  45,  46, ...,  91, 107, 180],
       [ 39,  43,  44, ...,  86, 109, 181],
       ...,
       [ 23,  23,  22, ...,  55,  68,  70],
       [ 21,  22,  21, ...,  58,  56,  64],
       [ 22,  22,  22, ...,  58,  57,  43]], dtype=uint8), array([1., 0.])]


In [40]:
training_data = np.load("training.npy", allow_pickle=True)

EOFError: No data left in file

In [66]:
array = np.array([[[1,2,3], [0,0]], [[4,5,6], [0,1]]], dtype=object)

In [67]:
array

array([[list([1, 2, 3]), list([0, 0])],
       [list([4, 5, 6]), list([0, 1])]], dtype=object)